# Generate Catan Boards

This notebook is used to generate randomised Catan boards.

In [1]:
from ipynb.fs.full.hex_grid import *
from ipynb.fs.full.draw_catan_board import *
from random import sample, seed
from statistics import variance
import numpy as np
import matplotlib.pyplot as plt
seed(2025)

In [11]:
# The board state is encoded as three tuples: types, tokens and harbour types.
# Each hex in the board is mapped to an index in the list (see hex_index_map below).
# Harbour locations aren't stored in the board state because they aren't randomised,
# and so it would be redundant to store it millions of times.

types = [2,1,5,1,4,1,3,2,3,0,5,2,1,2,4,5,4,3,3]
tokens = [5,6,9,11,9,8,2,4,8,0,11,6,3,4,10,5,10,12,3]
harbour_types = [0,5,1,0,0,3,2,0,4]
harbour_locations = [
    (Vertex(-1,3,1),Vertex(-1,2,0)),
    (Vertex(-2,2,0),Vertex(-3,3,1)),
    (Vertex(-3,2,1),Vertex(-2,0,0)),
    (Vertex(-2,0,1),Vertex(-1,-2,0)),
    (Vertex(0,-3,0),Vertex(0,-2,1)),
    (Vertex(1,-2,1),Vertex(2,-3,0)),
    (Vertex(2,-1,1),Vertex(3,-2,0)),
    (Vertex(3,-1,0),Vertex(2,1,1)),
    (Vertex(1,2,1),Vertex(1,1,0))
]

In [3]:
def shuffle(x):
    # random.shuffle is an in place method, use this instead
    return sample(x, len(x))

In [4]:
def shuffle_types(types: list[int], desert_center=True) -> list[int]:
    types = shuffle(types)
    if desert_center is True:
        desert_index = types.index(0)
        # Center token is always placed in the middle of the list
        center_index = (len(types)-1)//2
        types[desert_index], types[center_index] = types[center_index], types[desert_index]
    return types
    
def shuffle_tokens(tokens: list[int], types: list[int]) -> list[int]:
    tokens = shuffle(tokens)
    desert_index = types.index(0)
    zero_token_index = tokens.index(0)
    tokens[desert_index],tokens[zero_token_index] = tokens[zero_token_index],tokens[desert_index]
    return tokens
        
def shuffle_harbours(harbour_types: list[int]) -> list[list]:
    return shuffle(harbour_types)

def create_random_board(
    types: list[int],
    tokens: list[int],
    harbour_types: list[int],
    harbour_locations: list[list[Vertex]]) -> dict:
    
    board = {
    'types': shuffle_types(types),
    'tokens': shuffle_tokens(tokens, types),
    'harbour_types': shuffle_harbours(harbour_types)#,
    #'harbour_locations': harbour_locations
    }
    return board

def create_n_random_boards(
    types: list[int],
    tokens: list[int],
    harbour_types: list[int],
    #harbour_locations: list[list[Vertex]],
    n: int) -> list:
    boards = []
    for i in range(n):
        board = create_random_board(types, tokens, harbour_types, harbour_locations)
        boards.append(board)
    return boards

In [12]:
hex_grid = create_hexagonal_grid(2)
vertices = vertex_list(hex_grid)
board = create_random_board(types, tokens, harbour_types, harbour_locations)

# You can manually define a board like this 
# board = {
#    'types': [4,1,3,4,4,1,2,3,3,0,2,2,2,1,5,5,1,3,5],
#    'tokens': [6,10,8,9,5,4,10,3,12,0,6,2,3,9,8,5,11,11,4],
#    'harbour_types': [0,0,4,0,0,3,1,5,2]
#}

In [6]:
# These computations are relatively expensive.
# Instead of performing them over and over across millions of boards,
# hash the results instead.
hex_index_map = {}
for i in range(len(hex_grid)):
    hex_index_map[hex_grid[i]] = i

hex_neighbours_map = {}
for hex_ in hex_grid:
    neighbour_candidates = hex_neighbourhood(hex_)
    neighbours = []
    for n in neighbour_candidates:
        if n in hex_grid:
            neighbours.append(n)
    hex_neighbours_map[hex_] = neighbours

harbour_loc_index_map = {}
for i in range(len(harbour_locations)):
    harbour_loc_index_map[harbour_locations[i]] = i

vertex_to_hex_trip_map = {}
vertices = vertex_list(hex_grid)
for v in vertices:
    neighbour_candidates = vertex_to_hexes(v)
    neighbours = []
    for hex_ in neighbour_candidates:
        if hex_ in hex_grid:
            neighbours.append(hex_)
    vertex_to_hex_trip_map[v] = neighbours

In [10]:
write_boards = False
if __name__ == "__main__" and write_boards is True:
    n = int(1e7)
    file_name = "../data/boards_1e7.npy"
    boards = create_n_random_boards(types, tokens, harbour_types, n)
    np.save(file_name, boards)
